In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pyBigWig

In [3]:
data = pd.read_csv('../../../Anika/preprocessing/Overlapping_preprocessing_Carmen/preprocessing_results_genome/chr21_seq.tsv', sep='\t')
data

,#chr,start_pos,end_pos,1A,1C,1G,1T,2A,2C,2G,...,199T,200A,200C,200G,200T,201A,201C,201G,201T,target_C02M02
0,chr21,5010000,5010201,0,0,1,0,1,0,0,...,1,0,0,0,1,0,0,0,1,0.001680
1,chr21,5010025,5010226,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0.001680
2,chr21,5010050,5010251,0,1,0,0,0,1,0,...,1,0,0,1,0,0,0,1,0,0.001680
3,chr21,5010075,5010276,0,0,0,1,1,0,0,...,0,0,0,1,0,0,0,1,0,0.001680
4,chr21,5010100,5010301,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0.001680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1603126,chr21,46699525,46699726,0,1,0,0,0,1,0,...,1,0,0,0,1,0,1,0,0,0.008316
1603127,chr21,46699550,46699751,0,0,0,1,0,1,0,...,0,0,0,0,1,1,0,0,0,0.008316
1603128,chr21,46699575,46699776,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0.008316
1603129,chr21,46699600,46699801,0,1,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,0.008316


In [4]:
data_rm = data.iloc[:,-1:]

In [5]:
new_df = pd.concat([data.iloc[:,0:3],data.iloc[:,-1:]],axis = 1)

In [6]:
new_df.reset_index(drop=True, inplace=True) 

In [7]:
new_df.head()

,#chr,start_pos,end_pos,target_C02M02
0,chr21,5010000,5010201,0.00168
1,chr21,5010025,5010226,0.00168
2,chr21,5010050,5010251,0.00168
3,chr21,5010075,5010276,0.00168
4,chr21,5010100,5010301,0.00168


In [8]:
new_df = new_df.rename(columns = {"#chr": "chr"})

In [9]:
new_df.tail()

,chr,start_pos,end_pos,target_C02M02
1603126,chr21,46699525,46699726,0.008316
1603127,chr21,46699550,46699751,0.008316
1603128,chr21,46699575,46699776,0.008316
1603129,chr21,46699600,46699801,0.008316
1603130,chr21,46699625,46699826,0.008316


In [40]:
start_pos_list_replace = new_df["start_pos"].tolist()
start_pos_list_replace = start_pos_list_replace[1:] + [new_df["end_pos"].tolist()[-1]]

In [41]:
new_df_non_overlap = new_df
new_df_non_overlap["end_pos"] = start_pos_list_replace
new_df_non_overlap.tail()


,chr,start_pos,end_pos,target_C02M02
1603126,chr21,46699525,46699550,0.008316
1603127,chr21,46699550,46699575,0.008316
1603128,chr21,46699575,46699600,0.008316
1603129,chr21,46699600,46699625,0.008316
1603130,chr21,46699625,46699826,0.008316


In [62]:
bw = pyBigWig.open("bigwig_chr_21.bw", "w")
bw.addHeader([("chr21", 46709983)])

# chromosome lengths: https://www.ncbi.nlm.nih.gov/assembly/GCF_000001405.39

In [63]:
chrom_list = new_df_non_overlap["chr"].tolist()
chrom_list = np.array(["chr21"] * len(chrom_list)) 
start_pos_list = new_df_non_overlap["start_pos"].tolist()
end_pos_list = new_df_non_overlap["end_pos"].tolist()
target_list = new_df_non_overlap["target_C02M02"].tolist()
target_list[0:6]

for i in range(0, len(chrom_list)):
    bw.addEntries([chrom_list[i]], [start_pos_list[i]], ends=[end_pos_list[i]], values=[target_list[i]])

In [64]:
bw.close()

In [57]:
C02M02 = pyBigWig.open("bigwig_chr_21.bw")

In [58]:
C02M02.isBigWig()

True

In [59]:
C02M02.chroms()

{'chr21': 46699826}

In [61]:
C02M02.values("chr21", 5010000, 5010003)

[0.0016799999866634607, 0.0016799999866634607, 0.0016799999866634607]

In [80]:
import csv
new_df.chr = new_df.chr.astype(str)
new_df.to_csv('chr1.bed', index=False, quotechar="'",
                      header=None, quoting=csv.QUOTE_NONNUMERIC, sep = "\t")

check = pd.read_csv("chr1_tab.bed")
check
new_df.to_csv('chr1_tab.bed', index=False, header=None, sep = "\t")